In [1]:
import sqlite3
import re



In [2]:

# Connect to the SQLite database (it will create the file if it doesn't exist)
database_name = 'snow.db'  # Replace with your desired database file name
conn = sqlite3.connect(database_name)
cursor = conn.cursor()

In [3]:
create_tables = """
-- cmdb_ci Table
CREATE TABLE IF NOT EXISTS configuration_items (
    id VARCHAR(25) PRIMARY KEY,
    name TEXT,
    description TEXT,
    host TEXT,
    owner TEXT,
    upstream_ids TEXT,
    downstream_ids TEXT,
    created_date DATETIME,
    updated_date DATETIME
);

-- work_around Table
CREATE TABLE IF NOT EXISTS work_around (
    id VARCHAR(25) PRIMARY KEY,
    name TEXT,
    description TEXT,
    created_date DATETIME,
    last_updated_date DATETIME
);

-- known_problem Table
CREATE TABLE IF NOT EXISTS known_problem (
    id VARCHAR(25) PRIMARY KEY,
    name TEXT,
    description TEXT,
    cmdb_ci_id INTEGER,
    status TEXT,
    created_date DATETIME,
    updated_date DATETIME,
    FOREIGN KEY (cmdb_ci_id) REFERENCES cmdb_ci(id)
);

-- change_request Table
CREATE TABLE IF NOT EXISTS change_request (
    id VARCHAR(25) PRIMARY KEY,
    name TEXT,
    description TEXT,
    cmdb_ci_id INTEGER,
    severity TEXT,
    created_date DATETIME,
    updated_date DATETIME,
    FOREIGN KEY (cmdb_ci_id) REFERENCES cmdb_ci(id)
);

-- telemetry Table
CREATE TABLE IF NOT EXISTS telemetry (
    id VARCHAR(25) PRIMARY KEY,
    cmdb_ci_id INTEGER,
    name TEXT,
    description TEXT,
    data TEXT, -- Storing JSON data as TEXT
    created_date DATETIME,
    updated_date DATETIME,
    FOREIGN KEY (cmdb_ci_id) REFERENCES cmdb_ci(id)
);

-- logs Table
CREATE TABLE IF NOT EXISTS logs (
    id VARCHAR(25) PRIMARY KEY,
    incident_id INTEGER,
    name TEXT,
    description TEXT,
    data TEXT, -- Storing log entries as TEXT (could be JSON or other format)
    created_date DATETIME,
    updated_date DATETIME,
    FOREIGN KEY (incident_id) REFERENCES incident(id)
);

-- rca_document Table
CREATE TABLE IF NOT EXISTS rca_document (
    id VARCHAR(25) PRIMARY KEY,
    incident_id INTEGER,
    timeline_of_events TEXT,
    root_cause_analysis TEXT,
    contributing_factors TEXT,
    corrective_actions TEXT,
    follow_up TEXT,
    created_date DATETIME,
    updated_date DATETIME,
    FOREIGN KEY (incident_id) REFERENCES incident(id)
);

-- incident Table
CREATE TABLE IF NOT EXISTS incident (
    id VARCHAR(25) PRIMARY KEY,
    title TEXT,
    description TEXT,
    reported_by TEXT,
    status TEXT,
    priority TEXT,
    severity TEXT,
    assigned_group TEXT,
    assigned_to TEXT,
    resolution_notes TEXT,
    closure_code TEXT,
    closed_date DATETIME,
    incident_type TEXT,
    cmdb_ci_id INTEGER,
    work_around_id INTEGER,
    known_problem_id INTEGER,
    rca_document_id INTEGER,
    change_request_id INTEGER,
    telemetry_id INTEGER,
    logs_id INTEGER,
    created_date DATETIME,
    updated_date DATETIME,
    FOREIGN KEY (cmdb_ci_id) REFERENCES cmdb_ci(id),
    FOREIGN KEY (work_around_id) REFERENCES work_around(id),
    FOREIGN KEY (known_problem_id) REFERENCES known_problem(id),
    FOREIGN KEY (rca_document_id) REFERENCES rca_document(id),
    FOREIGN KEY (change_request_id) REFERENCES change_request(id),
    FOREIGN KEY (telemetry_id) REFERENCES telemetry(id),
    FOREIGN KEY (logs_id) REFERENCES logs(id)
);

-- kb_knowledge Table (Optional)
CREATE TABLE IF NOT EXISTS kb_knowledge (
    id VARCHAR(25) PRIMARY KEY,
    title TEXT,
    content TEXT,
    created_date DATETIME,
    updated_date DATETIME
);
"""

In [4]:


# Split the queries into individual statements
statements = create_tables.strip().split(';')

# Execute each SQL statement
for statement in statements:
    if statement.strip():  # Ensure the statement is not empty
        try:
            cursor.execute(statement)
        except sqlite3.Error as e:
            print(f"Error executing statement: {statement}\nError: {e}")

# Commit the changes to the database
conn.commit()


In [47]:
# # Replace problematic characters
# def sanitize_sql(sql_string):
#     # Replace problematic quotes in JSON strings
#     sanitized = re.sub(r'(\\")', r'\\\"', sql_string)
#     # Handle other special characters as needed
#     return sanitized

# # Split the queries into individual statements
# # statements = sql_queries.strip().split(';')


# # Apply to each statement before execution
# for statement in statements:
#     statement = statement.strip()
#     if statement:
#         try:
#             cursor.execute(sanitize_sql(statement))
#         except sqlite3.Error as e:
#             print(f"Error executing: {statement[:50]}...")
#             print(f"Error message: {e}")

# # Commit the changes to the database
# conn.commit()


In [48]:


# Close the connection
conn.close()

print(f"Data has been successfully inserted into the '{database_name}' database.")

Data has been successfully inserted into the 'snow.db' database.


In [5]:
import os
import csv
import sqlite3

def insert_csv_to_sqlite(folder_path, db_name):
    """
    Reads CSV files from a folder and inserts their data into a SQLite database.

    Args:
        folder_path (str): The path to the folder containing the CSV files.
        db_name (str): The name of the SQLite database file (e.g., 'mydatabase.db').
    """
    try:
        # Connect to the SQLite database (creates it if it doesn't exist)
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Get a list of all files in the folder
        csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

        if not csv_files:
            print(f"No CSV files found in the folder: {folder_path}")
            return

        for csv_file in csv_files:
            file_path = os.path.join(folder_path, csv_file)
            table_name = os.path.splitext(csv_file)[0]  # Extract table name from filename

            print(f"Processing file: {csv_file} (Table: {table_name})")

            with open(file_path, 'r', newline='') as csvfile:
                csv_reader = csv.reader(csvfile)
                header = next(csv_reader)  # Read the header row
                columns = [col.lower() for col in header] # Convert header to lowercase for case-insensitive matching
                if table_name.lower() == 'logs':
                    # Specific handling for logs.csv based on assumed column order
                    for row in csv_reader:
                        if len(row) == 7:
                            log_id, incident_id, name, description, data, created_date, updated_date = row
                            insert_sql = f"""
                                INSERT INTO logs (id, incident_id, name, description, data, created_date, updated_date)
                                VALUES (?, ?, ?, ?, ?, ?, ?)
                            """
                            try:
                                cursor.execute(insert_sql, (log_id, incident_id, name, description, data, created_date, updated_date))
                            except sqlite3.OperationalError as e:
                                print(f"Error inserting row into table 'logs': {e}")
                                print(f"Problematic row: {row}")
                            except sqlite3.IntegrityError as e:
                                print(f"IntegrityError inserting row into table 'logs': {e}")
                                print(f"Problematic row: {row}")
                        else:
                            print(f"Warning: Skipping row in 'logs.csv' due to incorrect number of columns: {row}")
                else:
                    # General handling for other tables using header matching
                    cursor.execute(f"PRAGMA table_info({table_name})")
                    table_info = cursor.fetchall()
                    db_columns = [info[1].lower() for info in table_info]
                    common_columns = [col for col in columns if col in db_columns]

                    if not common_columns:
                        print(f"Warning: No matching columns found between CSV header and table '{table_name}'. Skipping file.")
                        continue

                    placeholders = ', '.join(['?'] * len(common_columns))
                    insert_sql = f"INSERT INTO {table_name} ({', '.join(common_columns)}) VALUES ({placeholders})"

                    for row in csv_reader:
                        row_dict = dict(zip(columns, row))
                        values_to_insert = [row_dict.get(col) for col in common_columns]

                        try:
                            cursor.execute(insert_sql, values_to_insert)
                        except sqlite3.OperationalError as e:
                            print(f"Error inserting row into table '{table_name}': {e}")
                            print(f"Problematic row: {row}")
                        except sqlite3.IntegrityError as e:
                            print(f"IntegrityError inserting row into table '{table_name}': {e}")
                            print(f"Problematic row: {row}")



        # Commit the changes
        conn.commit()
        print("Data insertion completed successfully.")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except FileNotFoundError:
        print(f"Error: Folder not found at '{folder_path}'")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if conn:
            conn.close()


In [6]:

folder_path = 'data/snow'
db_name = 'snow.db'

insert_csv_to_sqlite(folder_path, db_name)

Processing file: rca_document.csv (Table: rca_document)
Processing file: incident.csv (Table: incident)
Processing file: work_around.csv (Table: work_around)
Processing file: change_request.csv (Table: change_request)
Processing file: logs.csv (Table: logs)
Processing file: kb_knowledge.csv (Table: kb_knowledge)
Processing file: telemetry.csv (Table: telemetry)
Processing file: known_problem.csv (Table: known_problem)
Processing file: configuration_items.csv (Table: configuration_items)
Data insertion completed successfully.
